# Mask R-CNN Training Demo

This notebook demonstrates the steps for training Mask R-CNN models. 
Mask R-CNN is a convolution-based neural network for the task of object instance segmentation. The original paper describing the model can be found [here](https://arxiv.org/abs/1703.06870). NVIDIA’s Mask R-CNN is an optimized version of [Facebook’s implementation](https://github.com/facebookresearch/maskrcnn-benchmark), leveraging mixed precision arithmetic and tensor cores for faster training times while maintaining comparable accuracy with single precision, i.e. FP32, training.

## Notebook  Content
1. [Pre-requisite: data](#1)
1. [Pre-requisite: container](#2)
1. [Training](#3)
1. [Testing trained model](#4)


<a id="2"></a>
## 1. Pre-requisite: data

This notebook demonstrates training and validation or the Mask R-CNN model on the [COCO 2014 dataset](http://cocodataset.org/#download). If not already available locally, the following [script](../../download_dataset.sh) in this repository provides a convinient way to download and extract all the necessary data in one go:

```
./download_dataset.sh <data/dir>
```

This script can be ran on a Linux host machine. Be mindful of the size of the raw data (~20GB). The script makes use of `wget` and will automatically resume if disrupted. Once downloaded, the scrip invoke `dtrx` to extract the data. The final data directory should look like:

```
<data/dir>
  annotations/
    instances_train2014.json
    instances_val2014.json
  train2014/
    COCO_train2014_*.jpg
  val2014/
    COCO_val2014_*.jpg
```

<a id="2"></a>
## 2. Pre-requisite: container

The most convinient way to make use of the NVIDIA Mask R-CNN model is via a docker container, which provides a self-contained, isolated and re-producible environment for all experiments. Follow the quick-start instruction at https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/Segmentation/MaskRCNN 

In short, first clone the repository:

```
git clone https://github.com/NVIDIA/DeepLearningExamples.git
cd DeepLearningExamples/PyTorch/Segmentation/MaskRCNN
```

Then build the Mask R-CNN PyTorch NGC container.

```
cd pytorch/
bash scripts/docker/build.sh
```